In [ ]:
# default_exp core

# module name here

> API details.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
from fastai2.basics import *

In [ ]:
#export
class TypeDispatch2(TypeDispatch):
    def __call__(self, *args, **kwargs):
        res = super().__call__(*args, **kwargs)
        if res is args[0]: return None
        return res

In [ ]:
#export
class DispatchReg2(DispatchReg):
    def __init__(self): self.d = defaultdict(TypeDispatch2)
typedispatch2 = DispatchReg2()

In [ ]:
#export
def reduce_lbls(lbls, abstain):
    lbls = lbls.filter(partial(equals, None), negate=True) # Remove all Nones
    if len(lbls)>1: raise ValueError # TODO: Should figure out this case
    return lbls[0] if len(lbls)==1 else abstain

In [ ]:
#export
def compose_tfms2(x, tfms, lfs, cats, is_enc=True, reverse=False, **kwargs):
    "Apply all `func_nm` attribute of `tfms` on `x`, maybe in `reverse` order"
    if reverse: tfms = reversed(tfms)
    lbls = L(L() for _ in range_of(lfs))
    for i,lf in enumerate(lfs): lbls[i].append(lf(x))
    for f in tfms:
        if not is_enc: f = f.decode
        x = f(x, **kwargs)
        for i,lf in enumerate(lfs): lbls[i].append(lf(x))
    # TODO: LF can be called twice (if types repeat)
    lbls = [reduce_lbls(v, cats.abstain) for v in lbls]
    return lbls

In [ ]:
#export
@delegates(Pipeline.__init__)
class Pipeline2(Pipeline):
    def __init__(self, funcs=None, lfs=None, categories=None, **kwargs):
        super().__init__(funcs=funcs, **kwargs)
        self.lfs,self.cats = lfs,categories
    def __call__(self, o): return compose_tfms2(o, tfms=self.fs, lfs=self.lfs,
                                                cats=self.cats, split_idx=self.split_idx)

Tests

In [ ]:
Cats = Enum('Cats', 'abstain fint fstr fflt flst')

In [ ]:
@typedispatch2
def fint(x:int): return Cats.fint
@typedispatch2
def fstr(x:str): return Cats.fstr
@typedispatch2
def fflt(x:float): return Cats.fflt
@typedispatch2
def flst(x:list): return Cats.flst

In [ ]:
@typedispatch
def flt2int(x:float): return int(x)
@typedispatch
def int2str(x:int): return str(x)
@typedispatch
def int2flt(x:int): return float(x)
@typedispatch
def flt2str(x:float): return str(x)
@typedispatch
def str2int(x:str): return int(float(x))

In [ ]:
tfms = Pipeline2([flt2str, str2int], lfs=[fint, fstr, fflt, flst], categories=Cats)

In [ ]:
test_eq(tfms(1.2), [Cats.fint, Cats.fstr, Cats.fflt, Cats.abstain])

## Export -

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted Untitled-Copy1.ipynb.
Converted index.ipynb.
Converted resume.ipynb.
